In [39]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
current_directory = os.getcwd()
print(current_directory) 



/Users/srite/repos/sri_avi_blog_posts/share_buybacks


In [76]:
### uncomment the following line if you dont have yfinance installed
#!pip install yfinance
#!pip install chart_studio
import yfinance as yf

import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.express as px



In [5]:
pd.set_option('display.max_rows', 500)

# This Analysis will feature a 2 part analysis. 

## Part 1 -- do buyback announcements directly influence share prices?
#### Things to Look at
- 5, 30, 90 day share price avg pre and post annoucement
- 
- sdfkj

## Part 1

### Lets take a look at how 2023 and 2024 sharebuyback announcements immediately impacted share price by doing a pre vs post analysis

In [43]:
buybacks = pd.read_csv('buyback_announcements_2023_2024.csv')
buybacks.columns = buybacks.columns.str.replace(' ', '_')
buybacks.columns = buybacks.columns.str.replace('-', '_')

buybacks.head(5)

,Symbol,Company,Current_Price,Price_Change,Date,Percent_of_Shares,Buyback_Amount,Offer_Type,Buyback_Type,52_Week_Low,52_Week_High,Indicator(s)
109,A,Agilent Technologies,$132.01,-2.50%,1/9/2023,4.40%,$2 billion,Open Market,New,$96.80,$145.43,Upcoming Earnings - News Coverage - Gap Down -
70,AAPL,Apple,$184.37,1.10%,5/4/2023,3.40%,$90 billion,Open Market,Additional,$143.90,$199.62,Analyst Revision -
26,ACM,AECOM,$89.23,1.30%,11/13/2023,8.30%,$1 billion,Open Market,NaN,$74.40,$94.01,Positive News -
48,ACN,Accenture,$371.27,2.90%,9/28/2023,2.00%,$4 billion,Open Market,Additional,$242.80,$375.73,Analyst Report - Analyst Revision - Gap Up -
8,AEO,American Eagle Outfitters,$22.96,2.30%,2/1/2024,NaN,30.00 million shs,Open Market,NaN,$9.95,$23.17,Gap Down -
32,AIZ,Assurant,$177.43,1.90%,11/9/2023,7.10%,$600 million,Open Market,Additional,$104.49,$179.41,Analyst Revision -
0,ALKS,Alkermes,$28.91,-0.90%,2/15/2024,8.20%,$400 million,Open Market,New,$22.01,$33.71,Earnings Report - Analyst Downgrade - Analyst ...
86,AMAT,Applied Materials,$199.73,4.90%,3/13/2023,9.70%,$10 billion,Open Market,Additional,$109.00,$206.77,Earnings Report - Analyst Report - Analyst Rev...
57,AMP,Ameriprise Financial,$403.31,2.00%,7/26/2023,9.50%,$3.50 billion,Open Market,Additional,$275.09,$403.75,Insider Selling - Analyst Revision -
60,ASH,Ashland,$93.02,0.90%,6/28/2023,21.20%,$1 billion,Open Market,New,$70.82,$105.15,NaN


In [44]:
#Get 5,30,90 day interval dates 
buybacks['Date'] = pd.to_datetime(buybacks['Date'])
buybacks['pre_date_5day'] = buybacks['Date']-timedelta(days=5)
buybacks['post_date_5day'] = buybacks['Date']+timedelta(days=5)
buybacks['pre_date_30day'] = buybacks['Date']-timedelta(days=30)
buybacks['post_date_30day'] = buybacks['Date']+timedelta(days=5)
buybacks['pre_date_90day'] = buybacks['Date']-timedelta(days=90)
buybacks['post_date_90day'] = buybacks['Date']+timedelta(days=90)


,Symbol,Company,Current_Price,Price_Change,Date,Percent_of_Shares,Buyback_Amount,Offer_Type,Buyback_Type,52_Week_Low,52_Week_High,Indicator(s),pre_date_5day,post_date_5day,pre_date_30day,post_date_30day,pre_date_90day,post_date_90day
109,A,Agilent Technologies,$132.01,-2.50%,2023-01-09,4.40%,$2 billion,Open Market,New,$96.80,$145.43,Upcoming Earnings - News Coverage - Gap Down -,2023-01-04,2023-01-14,2022-12-10,2023-01-14,2022-10-11,2023-04-09
70,AAPL,Apple,$184.37,1.10%,2023-05-04,3.40%,$90 billion,Open Market,Additional,$143.90,$199.62,Analyst Revision -,2023-04-29,2023-05-09,2023-04-04,2023-05-09,2023-02-03,2023-08-02
26,ACM,AECOM,$89.23,1.30%,2023-11-13,8.30%,$1 billion,Open Market,NaN,$74.40,$94.01,Positive News -,2023-11-08,2023-11-18,2023-10-14,2023-11-18,2023-08-15,2024-02-11
48,ACN,Accenture,$371.27,2.90%,2023-09-28,2.00%,$4 billion,Open Market,Additional,$242.80,$375.73,Analyst Report - Analyst Revision - Gap Up -,2023-09-23,2023-10-03,2023-08-29,2023-10-03,2023-06-30,2023-12-27
8,AEO,American Eagle Outfitters,$22.96,2.30%,2024-02-01,NaN,30.00 million shs,Open Market,NaN,$9.95,$23.17,Gap Down -,2024-01-27,2024-02-06,2024-01-02,2024-02-06,2023-11-03,2024-05-01
32,AIZ,Assurant,$177.43,1.90%,2023-11-09,7.10%,$600 million,Open Market,Additional,$104.49,$179.41,Analyst Revision -,2023-11-04,2023-11-14,2023-10-10,2023-11-14,2023-08-11,2024-02-07
0,ALKS,Alkermes,$28.91,-0.90%,2024-02-15,8.20%,$400 million,Open Market,New,$22.01,$33.71,Earnings Report - Analyst Downgrade - Analyst ...,2024-02-10,2024-02-20,2024-01-16,2024-02-20,2023-11-17,2024-05-15
86,AMAT,Applied Materials,$199.73,4.90%,2023-03-13,9.70%,$10 billion,Open Market,Additional,$109.00,$206.77,Earnings Report - Analyst Report - Analyst Rev...,2023-03-08,2023-03-18,2023-02-11,2023-03-18,2022-12-13,2023-06-11
57,AMP,Ameriprise Financial,$403.31,2.00%,2023-07-26,9.50%,$3.50 billion,Open Market,Additional,$275.09,$403.75,Insider Selling - Analyst Revision -,2023-07-21,2023-07-31,2023-06-26,2023-07-31,2023-04-27,2023-10-24
60,ASH,Ashland,$93.02,0.90%,2023-06-28,21.20%,$1 billion,Open Market,New,$70.82,$105.15,NaN,2023-06-23,2023-07-03,2023-05-29,2023-07-03,2023-03-30,2023-09-26


In [81]:
#Pull in share prices for dates in +- 90 days of Date using yfinance data


def fetch_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

stock_data = []

for index, row in buybacks.iterrows():
    ticker = row['Symbol']
    start_date = row['pre_date_90day']
    end_date = row['post_date_90day']
    announcement_date_price = row['Current_Price']
    buyback_date = row['Date']
    data_temp = fetch_stock_data(ticker, start_date, end_date)
    data_temp['Ticker'] = ticker
    data_temp['announcement_date'] = buyback_date;
    data_temp['announcement_date_price'] = announcement_date_price;
    
    stock_data.append(data_temp)

    
stock_data = pd.concat(stock_data)
stock_data = stock_data.reset_index()



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [86]:
stock_data['announcement_date_price'] = stock_data['announcement_date_price'].str.replace('$','')
stock_data['announcement_date_price'] = stock_data['announcement_date_price'].str.replace(',','')

stock_data['announcement_date_price'] = stock_data['announcement_date_price'].astype(float)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price
0,2023-11-17,23.610001,24.379999,23.500000,24.240000,24.240000,2341800.0,ALKS,2024-02-15,28.91
1,2023-11-20,24.100000,24.400000,23.730000,24.180000,24.180000,2548200.0,ALKS,2024-02-15,28.91
2,2023-11-21,24.170000,24.330000,23.750000,23.760000,23.760000,1310600.0,ALKS,2024-02-15,28.91
3,2023-11-22,23.850000,24.430000,23.850000,24.379999,24.379999,874400.0,ALKS,2024-02-15,28.91
4,2023-11-24,24.480000,24.740000,24.360001,24.389999,24.389999,468800.0,ALKS,2024-02-15,28.91
...,...,...,...,...,...,...,...,...,...,...
12570,2023-03-31,135.399994,138.729996,134.860001,138.339996,137.342087,2579800.0,A,2023-01-09,132.01
12571,2023-04-03,137.419998,138.089996,136.009995,137.860001,137.088516,1551800.0,A,2023-01-09,132.01
12572,2023-04-04,138.250000,139.110001,137.529999,138.210007,137.436554,1778700.0,A,2023-01-09,132.01
12573,2023-04-05,137.740005,139.270004,136.669998,138.089996,137.317215,2312100.0,A,2023-01-09,132.01


In [91]:
stock_data['deviation_from_announcement_date'] = (stock_data['Close'] - stock_data['announcement_date_price'])
stock_data['deviation_from_announcement_date_pct'] = round((stock_data['Close'] - stock_data['announcement_date_price'])/stock_data['Close'],3)



stock_data['within_5_days'] = 0;
stock_data['within_30_days'] = 0;
stock_data['within_90_days'] = 0;


stock_data.loc[abs( (stock_data['Date'] - stock_data['announcement_date']).dt.days) <= 5, 'within_5_days'] = 1
stock_data.loc[abs( (stock_data['Date'] - stock_data['announcement_date']).dt.days) <= 30, 'within_30_days'] = 1
stock_data.loc[abs( (stock_data['Date'] - stock_data['announcement_date']).dt.days) <= 90, 'within_90_days'] = 1
stock_data['announcement_offset'] = (stock_data['Date'] - stock_data['announcement_date']).dt.days

In [92]:
stock_data.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price,deviation_from_announcement_date,within_5_days,within_30_days,within_90_days,announcement_offset,deviation_from_announcement_date_pct
0,2023-11-17,23.610001,24.379999,23.500000,24.240000,24.240000,2341800.0,ALKS,2024-02-15,28.91,-4.670000,0,0,1,-90,-0.193
1,2023-11-20,24.100000,24.400000,23.730000,24.180000,24.180000,2548200.0,ALKS,2024-02-15,28.91,-4.730000,0,0,1,-87,-0.196
2,2023-11-21,24.170000,24.330000,23.750000,23.760000,23.760000,1310600.0,ALKS,2024-02-15,28.91,-5.150000,0,0,1,-86,-0.217
3,2023-11-22,23.850000,24.430000,23.850000,24.379999,24.379999,874400.0,ALKS,2024-02-15,28.91,-4.530001,0,0,1,-85,-0.186
4,2023-11-24,24.480000,24.740000,24.360001,24.389999,24.389999,468800.0,ALKS,2024-02-15,28.91,-4.520001,0,0,1,-83,-0.185


In [94]:
stock_data.loc[stock_data['Ticker'] == 'NVR',]

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,announcement_date,announcement_date_price,deviation_from_announcement_date,within_5_days,within_30_days,within_90_days,announcement_offset,deviation_from_announcement_date_pct
65,2023-11-16,6293.500000,6349.490234,6236.839844,6290.589844,6290.589844,25000.0,NVR,2024-02-14,7533.28,-1242.690156,0,0,1,-90,-0.198
66,2023-11-17,6332.970215,6332.970215,6253.520020,6298.000000,6298.000000,23000.0,NVR,2024-02-14,7533.28,-1235.280000,0,0,1,-89,-0.196
67,2023-11-20,6280.959961,6335.569824,6203.100098,6307.120117,6307.120117,22400.0,NVR,2024-02-14,7533.28,-1226.159883,0,0,1,-86,-0.194
68,2023-11-21,6290.009766,6344.910156,6239.370117,6244.220215,6244.220215,20500.0,NVR,2024-02-14,7533.28,-1289.059785,0,0,1,-85,-0.206
69,2023-11-22,6276.770020,6350.000000,6194.069824,6222.890137,6222.890137,12800.0,NVR,2024-02-14,7533.28,-1310.389863,0,0,1,-84,-0.211
70,2023-11-24,6196.009766,6232.830078,6143.229980,6195.060059,6195.060059,6700.0,NVR,2024-02-14,7533.28,-1338.219941,0,0,1,-82,-0.216
71,2023-11-27,6199.000000,6199.000000,6126.870117,6146.899902,6146.899902,14700.0,NVR,2024-02-14,7533.28,-1386.380098,0,0,1,-79,-0.226
72,2023-11-28,6148.000000,6150.520020,6088.720215,6128.660156,6128.660156,24400.0,NVR,2024-02-14,7533.28,-1404.619844,0,0,1,-78,-0.229
73,2023-11-29,6164.000000,6166.069824,6089.629883,6107.109863,6107.109863,22700.0,NVR,2024-02-14,7533.28,-1426.170137,0,0,1,-77,-0.234
74,2023-11-30,6107.109863,6159.540039,6052.580078,6155.390137,6155.390137,29900.0,NVR,2024-02-14,7533.28,-1377.889863,0,0,1,-76,-0.224


In [95]:
# 30 day chart

stock_data_30day = stock_data.loc[stock_data['within_30_days'] == 1,]
stock_data_30day.head(50)
fig = px.line(stock_data_30day, x="announcement_offset", y="deviation_from_announcement_date_pct", color='Ticker')
fig.show()


In [30]:
## read in yfinance data for stocks

aapl = yf.Ticker('AAPL')
aapl_df = aapl.history(period='1y')
#aapl_df['Close'].plot(title="APPLE's stock")
aapl_df = aapl_df.reset_index()
aapl_df



,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-02-23 00:00:00-05:00,149.293135,149.541808,146.458276,148.606796,48394200,0.00,0.0
1,2023-02-24 00:00:00-05:00,146.328931,146.408508,144.946312,145.931061,55469600,0.00,0.0
2,2023-02-27 00:00:00-05:00,146.925767,148.378007,146.667138,147.134644,44998500,0.00,0.0
3,2023-02-28 00:00:00-05:00,146.269261,148.288481,146.050427,146.627350,50547000,0.00,0.0
4,2023-03-01 00:00:00-05:00,146.050432,146.448302,144.240088,144.538498,55479000,0.00,0.0
5,2023-03-02 00:00:00-05:00,143.613439,145.931070,143.135977,145.135315,52238100,0.00,0.0
6,2023-03-03 00:00:00-05:00,147.253989,150.307696,146.547767,150.228119,70732300,0.00,0.0
7,2023-03-06 00:00:00-05:00,152.973464,155.470147,152.645229,153.013260,87558000,0.00,0.0
8,2023-03-07 00:00:00-05:00,152.883962,153.212211,150.327614,150.795120,56182000,0.00,0.0
9,2023-03-08 00:00:00-05:00,151.998686,152.655185,151.023893,152.058365,47204800,0.00,0.0
